# 日本語テキストの前処理

## 事前準備

READMEを参照

## トレーニング用データの読み込み

In [1]:
# トレーニング用データのフォルダ
data_dir = './data/'

# トレーニング用データのファイル名
text_file = 'raw.txt'

with open(data_dir + text_file, "r", encoding="utf-8") as f:
    text_list = f.readlines()

## データの正規化

正規化を行うための関数定義

使用させていただいた正規化のコード: https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja

In [2]:
# encoding: utf8
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]', '', s)  # remove tildes
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

読み込んだ文章に対して正規化を行う。

In [3]:
from tqdm.notebook import tqdm

# 前処理を全文に対して行う関数
def normalize_text(text_list):
    for i, line in enumerate(tqdm(text_list)):
        text_list[i] = normalize_neologd(line)


normalize_text(text_list)
print(text_list[:10])


['アンパサンド', '', 'アンパサンド(&、英語名:)とは並立助詞「…と…」を意味する記号である。ラテン語のの合字で、Trebuchet MSフォントでは、と表示され "et" の合字であることが容易にわかる。ampersa、すなわち "and per se and"、その意味は"and[the symbol which]by itself[is]and"である。', '', 'その使用は1世紀に遡ることができ、5世紀中葉から現代に至るまでの変遷がわかる。', 'Zに続くラテン文字アルファベットの27字目とされた時期もある。', '', 'アンパサンドと同じ役割を果たす文字に「のet」と呼ばれる、数字の「7」に似た記号があった(,U+204A)。この記号は現在もゲール文字で使われている。', '', '記号名の「アンパサンド」は、ラテン語まじりの英語「&はそれ自身 "and" を表す」(&per se and)のくずれた形である。英語以外の言語での名称は多様である。']


## トークナイズ
**MeCab**を使用してトークナイズ。

In [4]:
import MeCab

def tokenize_text(text_list):
    tagger = MeCab.Tagger('-Owakati -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
    tagger.parse('')
    for i, line in enumerate(tqdm(text_list)):
        parsed_line = tagger.parse(line)
        text_list[i] = parsed_line


tokenize_text(text_list)
print(text_list[:10])


['アンパサンド \n', '\n', 'アンパサンド (&、 英語名 :) と は 並立 助詞 「 … と … 」 を 意味 する 記号 で ある 。 ラテン語 の の 合字 で 、 Trebuchet MS フォント で は 、 と 表示 さ れ " et " の 合字 で ある こと が 容易 に わかる 。 ampersa 、 すなわち " and per se and "、 その 意味 は " and [ the symbol which ] by itself [ is ] and " で ある 。 \n', '\n', 'その 使用 は 1世紀 に 遡る こと が でき 、 5世紀 中葉 から 現代 に 至る まで の 変遷 が わかる 。 \n', 'Z に 続く ラテン文字 アルファベット の 27 字 目 と さ れ た 時期 も ある 。 \n', '\n', 'アンパサンド と 同じ 役割 を 果たす 文字 に 「 の et 」 と 呼ば れる 、 数字 の 「 7 」 に 似 た 記号 が あっ た (, U + 204 A )。 この 記号 は 現在 も ゲール文字 で 使わ れ て いる 。 \n', '\n', '記号 名 の 「 アンパサンド 」 は 、 ラテン語 まじり の 英語 「& は それ 自身 " and " を 表す 」(& per se and ) の くずれ た 形 で ある 。 英語 以外 の 言語 で の 名称 は 多様 で ある 。 \n']


## 前処理済みテキストデータの保存

In [5]:
# 保存ファイル名
output_file = 'train.txt'

with open(data_dir + output_file, 'w', encoding="utf-8") as f:
    f.writelines(text_list)
